In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

Introduction to Bootstrapping
=======

Estimating errors is something that on the surface seems extremely straightforward, but can in reality be a significant headache. In your labs, you've probably learned the theory behind error propagation, and while that works in a lot of cases, it assumes that you understand your errors in your measurements fully and that your errors are distributed normally. <b>This is almost never the case in astronomy!</b>

In the previous notebook, we found the errors in our measurements of $H_0$ by propogating the uncertainties in our measurements of $\mu$ in our polynomial fitting. In this notebook, we'll instead use bootstrapping. Before we get to the slopes, let's deal with a simpler example: combining tons of measurements of the Hubble constant that have already been made. To start, let's read in those measurements:

In [ ]:
data = pd.read_fwf('hubble_trim.dat',widths=[4,5,5,9,3,80],comment='#',
                     names=['h0','ep','em','date','method','source'],skiprows=1)

data

In the cell below, plot the measurements of H0, with errors, as a function of time. Plot with a log scale on the y-axis.

In [ ]:
plt.errorbar(data.date, data.h0, yerr=[np.abs(data.em), data.ep], fmt='b.')
plt.semilogy()
plt.xlabel('Date')
plt.ylabel(r'$H_0 (\frac{km/s}{Mpc})$')
plt.show()

We can see that the measurements have become significantly more consistent over time, narrowing down to around 70. Let's do a cut so that we plot just the data from 2001 onward and make the same plot, this time with a linear scale.

In [ ]:
data_modern = data.query('date>2001')

plt.errorbar(data_modern.date, data_modern.h0, yerr=[np.abs(data_modern.em), data_modern.ep], fmt='b.')
plt.xlabel('Date')
plt.ylabel(r'$H_0 (\frac{km/s}{Mpc})$')
plt.show()

As we can see, the errors aren't symmetric. Let's try plotting a histogram of the measurements, with the mean plotted on top as a dotted line.

In [ ]:
hdelta_mean = np.mean(data_modern.h0)

plt.hist(data_modern.h0, bins=np.arange(30,100, 2))
plt.axvline(hdelta_mean, ls='--', color='black')
plt.xlabel(r'$H_0 (\frac{km/s}{Mpc})$')
plt.show()

The bootstrap method is pretty simple: it's sampling with replacement. What we do is draw a random sample of size n from our sample m times, and then we compute the statistic we're after m times. These realizations let us get an idea of what it might look like if we had actually measured the sample m times, and as a result we can get a measure of the statistic that we would have measured. Let's do this for 10000 samples, drawing a sample that is the same length as our measured sample. To do this, we can use np.random.choice()

In [ ]:
ndata=len(data_modern.h0)
nbootstraps=int(1e4)

hboot=np.random.choice(data_modern.h0, size=(nbootstraps,ndata), replace=True)

np.shape(hboot)

hboot is now a 10000x134 array, with each row as a random draw from our distribution. The total distribution of this should match very closely with our original one for this many draws. Let's check that in the below cell, using density=True to normalize the histograms.

In [ ]:
plt.hist(data_modern.h0, density=True, histtype='step', bins=np.arange(30,100, 2), alpha=0.5)
plt.hist(np.ravel(hboot), density=True, histtype='step', bins=np.arange(30,100, 2), alpha=0.5)
plt.xlabel(r'$H_0 (\frac{km/s}{Mpc})$')
plt.show()

This matches perfectly, which is what we expect. Now, let's look at the first 5 draws instead:

In [ ]:
for i in range(0,5):
    plt.hist(hboot[i,:], density=True, histtype='step', bins=np.arange(30,100, 2), alpha=0.5)
    
plt.xlabel(r'$H_0 (\frac{km/s}{Mpc})$')
plt.show()

From these draws, we get different distributions that will measure different statistics. By measuring something like the mean multiple times, we can get a distribution of that statistic which will inform the bounds that we can put on that statistic.

In [ ]:
h0_mean = np.mean(hboot, axis=1)

print('Mean of the original distribution: ', np.mean(data_modern.h0))
print('Median of our bootstrap realizations: ', np.median(h0_mean))
plt.hist(h0_mean, bins=np.arange(64,70, 0.2))
plt.axvline(np.mean(data_modern.h0), ls='--', color='black')
plt.show()

This mean distribution seems to be relatively symmetric, but that doesn't necessarily need to be the case. One of the advantages of bootstrapping is that we can get asymmetric distributions of statistics and quantify confidence levels with percentiles. We can do that with np.percentile()

In [ ]:
print('2.5%, 16%, 50%, 84%, 97.5%: ', np.percentile(h0_mean, [2.5,16,50,84,97.5]))

In the cell below, find the distribution of the median using a bootstrapping method, plot a histogram of them, and get the 3-sigma upper and lower bounds on that measurement.

In [ ]:
h0_median = np.median(?

plt.hist(h0_median, bins=np.arange(64,73,0.2))
plt.show()

print('Upper and Lower 2-sigma values: ', np.percentile(?))

As you can see, the problem with the median here is that because our data is discrete, we don't really get a very evenly sampled set of measurements. One way to handle this is to add in noise to smooth out the measurements. Let's add in randomly sampled noise from a normal distribution. The magnitude of this noise is more of an art than a science; we just want it to be significantly smaller than the actual variance of the data. In this case, drawing from a normal distribution with sigma=1 is a reasonable thing to do.

In [ ]:
sboot = hboot + np.random.randn(nbootstraps,ndata)

h0_median_smoothed = np.median(sboot, axis=1)

plt.hist(h0_median_smoothed, bins=np.arange(64,73,0.2))
plt.show()

print('Upper and Lower 2-sigma values: ', np.percentile(?))

Simple statistics are very easy to recover with a method like this and are not very computational expensive. Now, let's return to the slope example from the previous notebook and fit $H_0$ with errors.

Costraining Errors in the Our Measurement from Last Session w/ Bootstrapping
======

In [ ]:
# CHANGE THE BELOW LINE TO POINT TO THE DIRECTORY CONTAINING SNDATA.TXT
path = ''  

# the pandas way: the file is in "fixed-width format" so we use read_fwf
data=pd.read_fwf(path+'sndata.txt')

cz=data['cz'] #already in km/s
logv = np.log10(data['cz'])
mu=data['mu']
sigma_mu=data['sigma_mu']
weight = 1/sigma_mu**2

coeffs, covar = np.polyfit(logv,mu,1,w=weight,cov=True)
slope_best = coeffs[0]
intercept_best = coeffs[1]
intercept_err_best = np.sqrt(covar[1,1])

def int_to_H0(b):
    return(10**(-0.2*b) * 10**5)

#fit for the best fitting H0 value and the symmetric error
h0_best = int_to_H0(intercept_best)
h0_best_err = (int_to_H0(intercept_best-intercept_err_best)-int_to_H0(intercept_best+intercept_err_best))/2

Now, let's synthesize everything we've learned. In the cell below, I want you to generate 10000 bootstrap samples of logv, mu, and sigma_mu. Assume a linear model for Hubble's Law and fit the slope and intercept for each of those realizations. Plot them against one another. Do you see any dependence between them? Where does the best fit value lie?

In [ ]:
nsims = int(1e4)
ndata = len(mu)

#generate 10000 samples, with replacement, of logv, mu, and weight. There are lots of ways to do this
rand_indices = np.random.randint(0, ndata, (nsims, ndata))

#initialize an array zeros for the h0 from the bootstrap methods
h0_boot = np.zeros(nsims)
intercept_arr = np.zeros(nsims)
slope_arr = np.zeros(nsims)

for i in range(0,nsims):
    #in this loop, populate h0_boot by finding the intercepts of our samples
    
    h0_boot[i] = 

plt.plot(intercept_arr, slope_arr, '.', color='blue')
plt.plot(intercept_best, slope_best, '*', color='red')
plt.xlabel('Intercept')
plt.ylabel('Slope')
plt.show()

Once you have the measurements of the intercept, convert them into a measurement of $H_0$ and plot a histogram of the values with the best fitting value as a dashed line. Are our measurements symmetric?

In [ ]:
#plot a histogram of the h0 values



Now, let's compare to the error we get from our polynomial fit. Print the best fitting value +/- the error we got from polyfit for that value, as well as the 16, 50, and 84th percentile values from our bootstrap realizations.

In [ ]:
print('Polyfit Values: ',[int_to_H0(intercept_best+intercept_err_best), 
                          h0_best, int_to_H0(intercept_best-intercept_err_best)])
print('Bootstrap Values: ', np.percentile(h0_boot, [16,50,84]))

As you should see, our median is very close to the value we get from our best fit, but the errors from our bootstrap samples capture the asymmtetry in the distribution and are larger than the original values. This type of error analysis is really useful for allowing us to capture those types of asymmetries and to account for covariances.

Jacknife Resampling
=======

Jackknife resampling is an older technique that's less widely used now because we have enough computational power to just bootstrap, but we can still go over it briefly. The way it works is that you remove one data point, make the measurement that you're going to make, replace the data point, remove another point, make the measurement, etc. until you've tested the data with every data point removed. The variance of the sample can be estimated from there as:

$\sigma_{jack}^2 ~=~ (n-1)*\sigma_{sample}^2$

Where $\sigma_{sample}^2$ is the variance of the measurements we make on the data with one point removed and $\sigma_{jack}$ is an estimate of the true variance. Jackknife methods are generally only useful when you don't have a lot of data points, and they don't really give us any way to estimate confidence intervals the way that bootstrap methods do, so they're not particularly useful for a data set like this. Nevertheless, let's compute them in the cell below.

In [ ]:
ndata = len(mu)

logv_jack = np.zeros((ndata, ndata-1))
mu_jack = np.zeros((ndata, ndata-1))
weight_jack = np.zeros((ndata, ndata-1))
h0_jack = np.zeros(ndata-1)

for i in range(0,ndata):
    logv_jack[i] = np.concatenate((logv[:i], logv[i+1:]))
    mu_jack[i] = np.concatenate((mu[:i], mu[i+1:]))
    weight_jack[i] = np.concatenate((weight[:i], weight[i+1:]))
    
h0_jack = np.zeros(ndata)

for i in range(0,ndata):
    #do the same fitting as above in this loop

    
    h0_jack[i] = int_to_H0(intercept)
    
plt.hist(h0_jack, bins=20)
plt.show()

h0_sigma_jack = np.std(h0_jack)*np.sqrt(ndata-1)

print('sigma jackknife, sigma_polyfit')
print(h0_sigma_jack, h0_best_err)

You'll see here that the jackknife resampling also gives a larger error than the error we get just from propogation of errors in our polynomial fit. It doesn't capture the asymmetry of our measurements, but it can still offer a more conservative estimate of our errors and can be useful for small data sets.